In [1]:
import requests 
from bs4 import BeautifulSoup

## Subject 1

In [ ]:
url = 'https://news.daum.net/economy'

def get_news(url, showDebug=True):
    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0'
    }

    res = requests.get(url=url, headers=req_header)

    if res.ok:
        soup = BeautifulSoup(res.content.decode('utf-8', 'replace'), 'html.parser') #한글 깨짐 방지
        # CSS 선택자
        news = soup.select("ul.list_newsheadline2 a[href*='v.daum.net/v']")
        if showDebug:
            print(url)
            print(type(res))
            print(res.status_code)
            print(type(news), len(news))
        for value in news:
            print(value.attrs["href"])
            print(value.text.strip())


    else:
        print(f"ERROR CODE: {res.status_code}")

get_news(url)


## Subject 1-2

In [ ]:

def print_news(section):
    section_dict = {'기후/환경':'climate','사회':'society','경제':'economy','정치':'politics',
                    '국재':'world','문화':'culture','생활':'life','IT/과학':'tech','인물':'people'}

    section_eng = section_dict[section]

    url = f"https://news.daum.net/{section_eng}"
    print(f"===> {url} {section_eng} 뉴스 <===")
    get_news(url, showDebug=False)
    




In [ ]:
print_news('경제')
print_news('사회')

## Subject 2

In [ ]:
from urllib.parse import urljoin
from IPython.display import Image, display

In [ ]:
url = "https://news.nate.com/recent?mid={0}"
sections = {
    "최신": "n0100",
    "정치": "n0101",
    "경제": "n0102",
    "사회": "n0103",
    "세계": "n0104",
    "IT/과학": "n0105"
}


def getNateNews(sectionName):
    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0'
    }

    res = requests.get(url=url.format(sections[sectionName]), headers=req_header)

    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser') #한글 깨짐 방지
        # CSS 선택자
        news = soup.select(".postSubjectContent a[href*='//news.nate.com/view']")
        for value in news:
            
            hrefUrl = urljoin(base=url, url=value.attrs.get("href"), allow_fragments=True)
            if iamgeUrl := value.find("img"):
                imageUrl = urljoin(base=url, url=iamgeUrl.attrs.get("src"), allow_fragments=True)
                display(Image(url=imageUrl))
            print(value.find("h2").text)
            print(hrefUrl)


    else:
        print(f"ERROR CODE: {res.status_code}")

getNateNews("정치")

## Subject 2-2

In [ ]:
import os 

def download_one_episode(title,no,url):
    if not os.path.exists(dirPath := os.path.join("img", title, str(no))):
        os.makedirs(dirPath)
    else:
        req_header = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0'
        }

        res = requests.get(url=url, headers=req_header)

        if res.ok:
            soup = BeautifulSoup(res.text, 'html.parser') 
            # CSS 선택자
            contents = [data.get("src") for data in soup.select(".wt_viewer img")]
            for i,value in enumerate(contents):
                imgRes = requests.get(value, headers=req_header)
                if imgRes.ok:
                    with open(os.path.join(dirPath,str(i))+".jpg", 'wb') as file:
                        img_data = imgRes.content
                        file.write(img_data)
                else:
                    print(f'Error Code = {imgRes.status_code} for {value}')


        else:
            print(f"ERROR CODE: {res.status_code}")

download_one_episode('낢이사는이야기',48,'https://comic.naver.com/webtoon/detail?titleId=833255&no=49&week=tue')

## Subject 3-1 

In [2]:
from dotenv import load_dotenv
import os 

load_dotenv(dotenv_path=".env", override=True, verbose=True)

def getApiKey(apiName):
    try:
        key = os.getenv(apiName)
    except:
        raise Exception("No Api Key!")
    
    return key

In [ ]:
from datetime import datetime
import json
import pandas as pd

def FormBookOutput(title,datas):
    print(f"=== {title} ===")
    for data in datas:
        print(f"""
    {data.get("title")}
    저자: {data.get("author")}
    출판사: {data.get("publisher")}
    출판일: {datetime.strptime(data.get("pubdate"), "%Y%m%d").strftime("%Y-%m-%d")}
    가격: {data.get("discount")}
    설명: {des[:101]+"..." if len(des := data.get("description"))> 100 else des}
        """
        )


def search_books(query):
    url = 'https://openapi.naver.com/v1/search/book.json'
    client_id = getApiKey("NAVER_CLIENT_KEY")
    client_secret = getApiKey("NAVER_SECRET_KEY")


    req_header = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    payload = {
        'query':  query, #'파이썬',
        'display': 30,
        'sort': 'sim'
    }

    res = requests.get(url, params=payload, headers=req_header)
    return res.json()["items"]

resultList = search_books("파이썬")

# 1. 질문 :  검색어로  찾은  책 목록 출력하기
# FormBookOutput("도서 목록", resultList)
# 2. 질문 :  검색어로  찾은  책 목록 중에서 가격이 2만원 이상인 책만 출력하기
# FormBookOutput("가격이 20,000원 이상인 도서 목록", list(filter(lambda x:int(x.get("discount"))>=20000, resultList)))
# 3. 질문 :  검색어로  찾은  책 목록 중에서 출판사가 인피니티북스인 책만 출력하기
# FormBookOutput("인피니티북스 출판 도서 목록", list(filter(lambda x:x.get("publisher")=="인피니티북스", resultList)))

# 4. 질문 :  검색어로  찾은  책 목록을 json 파일로 저장하기
with open("./data/books.json", "w", encoding="utf-8") as f:
    json.dump(resultList, f, ensure_ascii=False, indent=2)
# 2. books.json 파일을 Pandas DataFrame로 저장하기
df = pd.read_json("./data/books.json")

# 3. 질문 :  검색어로  찾은  책 목록 출력하기
print(df)

# 4. 질문 :  검색어로  찾은  책 목록 중에서 가격이 2만원 이상인 책만 출력하기
print(df[df['discount'] >= 20000][["title","author", "discount", "publisher", "pubdate"]].sort_values(by="discount", ascending=False, axis=0).reset_index(drop=True))

# 6. 질문 :  검색어로  찾은  책 목록 중에서 출판사가 인피니티북스인 책만 출력하기
print(df[df['publisher'] == "인피니티북스"][["title","author", "discount", "publisher", "pubdate"]].reset_index(drop=True))

     title author  discount publisher   pubdate
0      파이썬    천인국     12000    인피니티북스  20170830
1  파이썬 플러스    최희식     23750    인피니티북스  20240731


## Subject 3-2

In [ ]:
def FormShopOutput(title,datas): # f"{...:,}" => 숫자에 천 단위마다 ,를 넣으라는 뜻
    print(f"=== {title} ===")
    for data in datas:
        print(f"""
    {data.get("title")}
    최저가: {int(data.get("lprice")):,}원  
    브랜드: {data.get("brand")}
    쇼핑몰: {data.get("mallName")}
    링크: {data.get("link")}
        """
        )


def search_shops(query):
    url = 'https://openapi.naver.com/v1/search/shop.json'
    client_id = getApiKey("NAVER_CLIENT_KEY")
    client_secret = getApiKey("NAVER_SECRET_KEY")


    req_header = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    payload = {
        'query':  query, #'파이썬',
        'display': 30,
        'sort': 'sim'
    }

    res = requests.get(url, params=payload, headers=req_header)
    return res.json()["items"]


resultList = search_shops("가디건")

# 2. shops.json 파일을 Pandas DataFrame로 저장하기
with open("./data/shops.json", "w", encoding="utf-8") as f:
    json.dump(resultList, f, ensure_ascii=False, indent=2)
df = pd.read_json("./data/shops.json")

# 3. 질문 :  검색어로  찾은  Shop의 상품  목록 출력하기
# FormShopOutput("쇼핑 검색 결과", resultList)

# 2. 질문 :  검색어로  찾은  Shop의 상품  목록 중에서 가격이 50,000원 이하인 상품만 출력하기
# FormShopOutput("50,000원 이하 상품", list(filter(lambda x:int(x.get("lprice")) <= 50000, resultList)))

# 4. 질문 :  검색어로  찾은  Shop의 상품  목록 중에서 특정 쇼핑몰 상품만 출력하기
# FormShopOutput("네이버 상품", list(filter(lambda x:x.get("mallName") == "네이버", resultList)))


# 3. 질문 :  검색어로  찾은  Shop의 상품  목록 출력하기
print(df)

# 4. 질문 :  검색어로  찾은  Shop의 상품  목록 중에서 가격이 50,000원 이하인 상품만 출력하기
print(df[df['lprice'] <= 50000][["brand","lprice", "mallName", "link"]].sort_values(by="lprice", axis=0).reset_index(drop=True))

# 4. 질문 :  검색어로  찾은  Shop의 상품  목록 중에서 특정 쇼핑몰 상품만 출력하기
print(df.loc[:,'lprice':'brand'].sort_values(by="lprice", axis=0).reset_index(drop=True))
